# CODEATHON 2: Recognizing UVA landmarks with neural nets (50 pts)
![UVA Grounds](https://giving.virginia.edu/sites/default/files/2019-02/jgi-teaser-image.jpg)

The UVA Grounds is known for its Jeffersonian architecture and place in U.S. history as a model for college and university campuses throughout the country. Throughout its history, the University of Virginia has won praises for its unique Jeffersonian architecture.

In this codeathon, you will attempt the build an image recognition system to classify different buildlings/landmarks on Grounds. You will earn 50 points for this codeathon plus 10 bonus points. To make it easier for you, some codes have been provided to help you process the data, you may modify it to fit your needs.

You must submit the .ipynb file via UVA Collab with the following format: **yourUVAComputingID_codeathon_2.ipynb**

In [117]:
"""
Note to self: I will learn how to vallidate machien learning workflows with tthe simple model first approach --> helps with baseline

- Get a specific focus on whether I know and verify mentally key representaations
- can i use tools cororectly in order to build in tension to me wanting to build as fast as possible?
- think about specific long term dependencies that might happen especiaalyl with coding and make practices now that will prevent the error - esp with learning 
0 jupyter note book + library hybrid for better communication.

- see how rigorousness contradicts with the perceived amount of time -> I decide to be too rigorous, does this actually reduce the amount of time i have from later debuggin?

- See methods for debugging and verification of the dataset + form attributes



- Representational issue - Representing Batch Sizes + Relation in TF to othe training --> Why is f1 score going wronig? 
- how do we know that the fit functions for CNN are the same with the data? --> data alignment problem.
- Rep 1 - What is the dataset tf outputting? --> categorical labels or one hot encodings
- in representation of code - i dont know "hwat is happenign" nor can i anticipate the representations. 
---> here somehow in achieving a better representation I can heuristically analyze by removing complexity and seeing if any key interactions take place.


- i really want to add the recall metrics to tensorflow -- there is some sort of internal error with the interface - some sort of interaction between the dataset model, and recall fucntion is happening.
    - problem 1 - tf dataset is hard to represent rather than using explicit tensors.

# Relataoin to other types of learnign tasks 
- here it is much easier to slow down and step back as I can always expand my time to fit the cogntive  load, there are no time dependencies 

    


# Problems I solved

1. I establish a baseline model - "this is the refeerence point i have bad performance" - > i want to know if changing model parameters actually does welll and incrases performacne --> identifies changes in model to outcomes"
    here convergence also informs aabout hyperparametres that need to change in context to the goal of minimized time. - bang for your buck convergence.
3. How do I monitor how well the model is learning and converging? --> how do i interpert plateauing performance?
4. Can i implement flows that dyanmically adjust epoochs for convergence? like when does it stop?
2. Metrics - How can I fit the code so taht it can explain to me how well hte model is doing?
    - idea w precisoin + recall - we need probabilities NOT logits, this had to do with a model output problem.

4. sB1 - How doo i find errors in a mach8ien learning pipeline? --> I want to simplify the dataset and attributes and observe interface wise what is happening - devise ways of understanding
    - use git to "expierment" instead of creating feature --> time spent - 1hr of learning 

"""


import sys
import sklearn
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from functools import partial

#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

from logging_practice import setup_logger, log_function


np.random.seed(42)
tf.random.set_seed(42)

# Step 1: Process the  Dataset
The full dataset is huge (+37GB) with +13K images of 18 classes. So it will take a while to download, extract, and process. To save you time and effort, a subset of the data has been resized and compressed to only 379Mb and stored in my Firebase server. This dataset will be the one you will benchmark for your grade. If you are up for a challenge (and perhaps bonus points), contact the instructor for the full dataset!

In [118]:
# Download dataset from Firebase
#!wget https://firebasestorage.googleapis.com/v0/b/uva-landmark-images.appspot.com/o/dataset.zip?alt=media&token=e1403951-30d6-42b8-ba4e-394af1a2ddb7

In [119]:
# Extract content
#TODO: Change this back so that powershell version is desired
#!unzip "/content/dataset.zip?alt=media"

!powershell Expand-Archive -Path dataset.zip -DestinationPath .

New-Object : Exception calling ".ctor" with "3" argument(s): "End of Central Directory record could not be found."
At 
C:\WINDOWS\system32\WindowsPowerShell\v1.0\Modules\Microsoft.PowerShell.Archive\Microsoft.PowerShell.Archive.psm1:934 
char:23
+ ... ipArchive = New-Object -TypeName System.IO.Compression.ZipArchive -Ar ...
+                 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (:) [New-Object], MethodInvocationException
    + FullyQualifiedErrorId : ConstructorInvokedThrowException,Microsoft.PowerShell.Commands.NewObjectCommand
 


In [128]:



from sklearn.datasets import load_files
#from keras.utils import np_utils

#from keras.preprocessing import imag

from tqdm import tqdm # progress bar



""" 
TODO: 

Learn more rigoroulsy about the code why certain processes happen / debugs
---> Learna exactly the behavior of my model --> use those insights to - i want the deiling here to be as high as possibel for model interpretation.
Simplify + Log the interactions
- provide methods if simplification 
- provide methods of rigorous logging - better than print statements and I can turn off and on.
- other contexts - printing the losses, visualization of results.\\

- later on use / review the code clutttering while - interfacing andd logging other actions - consider ideas of decorators
-

"""


import logging

logger = setup_logger(level = logging.INFO)



@log_function(logger, logging.INFO)
def create_dummy_tf_datasets(hyperparameters):
    # Set seeds for reproducibility
    tf.random.set_seed(42)
    np.random.seed(42)


    batch_size, img_height, img_width, n_channels, n_classes, n_total_images = hyperparameters


    
    # Create dummy images and labels
    # Using random values between 0 and 255 for images
    dummy_images = tf.random.uniform(
        shape=(n_total_images, img_height, img_width, n_channels),
        minval=0,
        maxval=255,
        dtype=tf.float32
    )
    
    # Create dummy labels (alternating between 0 and 1)
    dummy_labels = tf.constant([i % n_classes for i in range(n_total_images)])
    
    # Split into training and validation
    # Training: 8 images (80%), Validation: 2 images (20%)
    train_size = int(0.8 * n_total_images)
    
    # Split the data
    train_images = dummy_images[:train_size]
    train_labels = dummy_labels[:train_size]
    val_images = dummy_images[train_size:]
    val_labels = dummy_labels[train_size:]
    
    # Create datasets
    train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    validation_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
    
    # Configure datasets to match tf.keras.preprocessing.image_dataset_from_directory behavior
    # Batch, shuffle, and repeat
    train_ds = train_ds.shuffle(buffer_size=1000, seed=42)
    train_ds = train_ds.batch(batch_size)
    train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
    
    validation_ds = validation_ds.batch(batch_size)
    validation_ds = validation_ds.prefetch(tf.data.AUTOTUNE)
    
    # Add class_names attribute to match directory-loaded dataset behavior
    class_names = ['class1', 'class2']
    train_ds.class_names = class_names
    validation_ds.class_names = class_names
    
    return train_ds, validation_ds


# TODO: Change this back to content/dataset
data_dir = "./dataset/"


batch_size = 2
img_height = 30
img_width = 30
n_channels = 2
num_classes = 2
n_total_images = 10  # Total number of images


hyperparameters = [batch_size, img_height, img_width, n_channels, num_classes, n_total_images]


# Dataset that loads the specific portion using "subset" parameter


# Training Dataset
train_ds, validation_ds = create_dummy_tf_datasets(hyperparameters)


# Convert the labels into one hot encodings 

#train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=18)))
#validation_ds = validation_ds.map(lambda x, y: (x, tf.one_hot(y, depth=18)))


print(train_ds)
print(validation_ds)




AttributeError: 'int' object has no attribute 'info'

In [ ]:
type(train_ds)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [ ]:
# YOUR CODE STARTS HERE

# Step 2: Create your own CNN architecture
You must design your own architecture. To get started, you may get inspiration from one in CNN notebook  (i.e. use one similar to LeNet-5 or AlexNet). You will have to report the design of the architecture:

1.   How many layers does it have?
2.   Why do you decide on a certain number nodes per layer?
3.   Which activation functions do you choose?
4.   How many parameters does it has in total?

Hint: use `myModel.summary()` to learn on the layers and parameters




In [ ]:
# Your code here
# Design a Model that does really welll in classifying images
# Clearly a CNN architectures is useful here because we dont reallly need to process long term dependencies in order to build the building featuresa
# Use a baseline model + hyperparameters to make a baseline evaluation --> establishes a reference point for showing if it is performinb well.



myModelBaseLine = keras.models.Sequential([
    # First conv-pool block
    keras.layers.Conv2D(1, (1, 1), activation='relu', input_shape=(img_height, img_width, n_channels)), # idea 1 - input shape must reflect batch sizes
    keras.layers.MaxPooling2D(1, 1),
    
    # Second conv-pool block
    keras.layers.Conv2D(1, (1, 1), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
    
    # Third conv-pool block
    keras.layers.Conv2D(1, (1,1), activation='relu'),
    keras.layers.MaxPooling2D(2,2),

    # Flatten and dense layer
    keras.layers.Flatten(),
    keras.layers.Dense(num_classes, activation = 'softmax'), # No activation for logits

])

"""

# Third conv-pool block
    keras.layers.Conv2D(256, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(2,2),
# Third conv-pool block
keras.layers.Conv2D(512, (3,3), activation='relu'),
keras.layers.MaxPooling2D(2,2),
""" 
myModelBaseLine.summary()


"""
I have created a CNN network that contains 3 layers of convolutions and pooling foolowed by a Dense Linear layer, that outputs logits for a class. For the number of nodes per layer, I indirectly decidedd the number of nodes per layer
throuogh teh kernel size of (3,3) because I thought that heuristically that this was a reasonable starting point for helping the model process local features without being overwhelmed by a large kernel size. I decided to use a RELU function
because this was computationally easy to compute. ReLU fucntionos 

My model has a total of 1,605,458 parameters


"""

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_75 (Conv2D)              │ (None, 30, 30, 1)      │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_75 (MaxPooling2D) │ (None, 30, 30, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_76 (Conv2D)              │ (None, 30, 30, 1)      │             2 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_76 (MaxPooling2D) │ (None, 15, 15, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_77 (Conv2D)              │ (None, 15, 15, 1)      │             2 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_77 (MaxPooling2D) │ (None, 7, 7, 1)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_21 (Flatten)            │ (None, 49)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 2)              │           100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 107 (428.00 B)

 Trainable params: 107 (428.00 B)

 Non-trainable params: 0 (0.00 B)

'\nI have created a CNN network that contains 3 layers of convolutions and pooling foolowed by a Dense Linear layer, that outputs logits for a class. For the number of nodes per layer, I indirectly decidedd the number of nodes per layer\nthrouogh teh kernel size of (3,3) because I thought that heuristically that this was a reasonable starting point for helping the model process local features without being overwhelmed by a large kernel size. I decided to use a RELU function\nbecause this was computationally easy to compute. ReLU fucntionos \n\nMy model has a total of 1,605,458 parameters\n\n\n'

After designing the model, you will need to train it. In order to train, you will need to pick a number of `epoch` (iteration), which `optimizer` to use (from  `keras.optimizers`), a `loss` function, and some `metrics`.

In [ ]:
myEpochs = 8 # can i craet a dynamic convergence of this?
learning_rate = 0.001


# Complexities --> is this right "thing" that is used to optimimze?
myOptimizer = tf.keras.optimizers.Adam(
    learning_rate=learning_rate,
    name='Adam',
)
myLoss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)
myMetrics = myMetrics = [
    'accuracy',  
    tf.keras.metrics.Precision(),                                                                                                                    # Standard accuracy
]


    

myModelBaseLine.compile(loss= myLoss, optimizer = myOptimizer, metrics = myMetrics)
history = myModelBaseLine.fit(train_ds,
                      validation_data = validation_ds,
                      epochs = myEpochs)

Epoch 1/8


InvalidArgumentError: Graph execution error:

Detected at node LogicalAnd_1 defined at (most recent call last):
  File "c:\tools\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main

  File "c:\tools\Anaconda3\lib\runpy.py", line 87, in _run_code

  File "c:\tools\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\tools\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance

  File "c:\tools\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start

  File "c:\tools\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start

  File "c:\tools\Anaconda3\lib\asyncio\base_events.py", line 601, in run_forever

  File "c:\tools\Anaconda3\lib\asyncio\base_events.py", line 1905, in _run_once

  File "c:\tools\Anaconda3\lib\asyncio\events.py", line 80, in _run

  File "c:\tools\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 510, in dispatch_queue

  File "c:\tools\Anaconda3\lib

Next, you need to create (1) a plot of training and validation `loss` and (2) a plot of training and validation `accuracy`. These plots might give you some insights about your model performance and possibility of overfitting.

Report the performance of your architecture on the validation set in a `confusion matrix`. Make comments on the performance by answering the following questiosns:
- How well do you think your architecture is doing (overall accuracy)?
- Where did it makes mistake most?
- Which classes can be improved?

In [ ]:
# Your evaluation code here

# Step 3: Use a Pre-trained Network with Transfer Learning
Now that you have a your own custom model and some baseline performance, let's see if you can improve the performance using transfer learning and a pre-trained model. You may use any pre-trained model EXCEPT ones that already provided such as `Xception`, `MobileNet`, `EfficientNetB6`. Keep in mind that each pre-trained model may expect a different input shape, so adjust the size of your training images accordingly.

Make sure you report the design of this architecture by answer the same questions 1-4 in Step 3.

Hint: use `ImageNet` as weights when load the pre-train network, then add a `GlobalAveragePooling2D` and an output layer with `softmax` activation.



In [ ]:
# Your code here

Next, you will attempt to adapt this pre-trained model to your UVA Landmark dataset. It is recommended that you tried the two-phase training approach for your model:

1.   Phase 1: Freeze the pre-train weights and only train the top layer
2.   Phase 2: Train the entire network with much smaller learning rate (adapt the model to UVA data, but avoid destroying the transfered weights).



In [ ]:
# Phase 1 code here

In [ ]:
# Phase 2 code here

Repeat the same reporting of performance using the confusion matrix:
- Did this pre-trained network do better overall?
- In which class it improve the accuracy from the above model?
- Which class still has low performance?

Typically, your network must have a reasonable performance of at least 84% overall accuracy to be considered successful in this domain. If your network achieves a accuracy of 94% or above on the validation set, you will also recieve a 10 bonus points, so keep trying!

# Step 4: Reflection

Write at least a paragraph answering these prompts: How did your own network perform in comparison to the pre-trained one? What are the major differences between the architectures? Additionally, report on your experience implementing different models for this assignment (Was it hard/easy/fun?, from which part did you learn the most?)!